# Загрузка модели и подготовка сабмита

In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import torch
import torch.nn.functional as F
import torch.optim as optim


from datetime import datetime
from torch import nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import MessagePassing, DataParallel
from torch_scatter import scatter
from torch.utils.tensorboard import SummaryWriter

from DataClasses import lmdb_dataset, Dataset, DataListLoader

В зависимости от модели в этой клетке будут разные импорты

In [2]:
import sys
sys.path.append(os.path.expanduser('../../ocp_airi/ocpmodels/models'))
sys.path.append(os.path.expanduser('../../ocp_airi/'))

#from spinconv_with_solid_angles import spinconv
from spinconv_with_embeds_single import spinconv

Для каждой модели надо написать свою фукнцию preproceccing(), но обязательно, чтобы в массиве keys был элемент 'sid'

In [3]:
#вызывается каждый раз, когда датасет отдаёт элемент (систему)
#делаем из данных матрицу векторов-атомов, список рёбер (edge_index) и матрицу векторов-рёбер; надо писать свою функцию для каждой сети
def preprocessing(system):
    # keys = ['sid', 'pos', 'atomic_numbers', 'cell', 'natoms']
    # features_dict = {}
    # for key in keys:
    #     features_dict[key] = system[key]
    return system

Функция, которая расчитывает сабмит

In [4]:
def inference(model, iterator):
    y = torch.tensor([])
    model.eval()

    with torch.no_grad():
        for index, data_list in enumerate(tqdm(iterator)):
            sids = torch.tensor([[data.sid] for data in data_list])
            predictions = model(data_list).squeeze()
            predictions = predictions.reshape((predictions.shape[0], 1)) 

            mini_submit = torch.cat((sids, predictions.to('cpu')), dim=1)
            y = torch.cat((y, mini_submit))

    return y

In [5]:
batch_size = 40
num_workers = -1

target_col = 'y_relaxed'
features_cols = ['feature_1']

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
print(device)

cuda


Здесь мы задаем адрес чекпоинта модели, надо вставить адрес чекпоинта

In [14]:
# адрес датасета
test_dataset_path = "../../ocp_datasets/data/is2re_test_challenge_2021/data.lmdb"
# test_dataset_path = "../../ocp_datasets/data/is2re_test_challenge_2021/data_mod1.lmdbz"
# test_dataset_path = "../../ocp_datasets/data/is2re/10k/train/data_mod1.lmdbz"

# адрес чекпоинта модели
model_adress = "2021-10-05-14-31-19_epoch_8_.pickle"

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [15]:
train_dataset_file_path = os.path.expanduser(test_dataset_path)

In [16]:
train_dataset_file = lmdb_dataset(train_dataset_file_path)
# train_dataset_file.iloc(64509, 64517)
len(train_dataset_file)

120000

In [17]:
train_dataset_file.describe(5)

total entries: 120000
info for item: 5
edge_index:...............<class 'torch.Tensor'>..... [2, 3368]
pos:......................<class 'torch.Tensor'>.....   [78, 3]
cell:.....................<class 'torch.Tensor'>..... [1, 3, 3]
atomic_numbers:...........<class 'torch.Tensor'>.....      [78]
natoms:...................       <class 'int'>.....        78
cell_offsets:.............<class 'torch.Tensor'>..... [3368, 3]
distances:................<class 'torch.Tensor'>.....    [3368]
fixed:....................<class 'torch.Tensor'>.....      [78]
sid:......................       <class 'int'>.....     10039
tags:.....................<class 'torch.Tensor'>.....      [78]


In [18]:
training_set = Dataset(train_dataset_file, features_cols, target_col, preprocessing=preprocessing, type_='test')
training_generator = DataListLoader(training_set, batch_size=batch_size)

В этой клетке надо раскомментить первую строчку и закомментить вторую

In [19]:
# если модель сохраняется через state_dict, то процедура загрузки модели отличается
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
model = torch.load(model_adress)
# model = spinconv(None, None, 1, otf_graph=True, regress_forces=False)

#переносим на куду если она есть
model = model.to(device)

In [20]:
%%time

# print(f'Start training model {str(model)}')
submission = inference(model, training_generator)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [23:24<00:00,  2.14it/s]

CPU times: user 5h 31min 10s, sys: 3h 45min 36s, total: 9h 16min 47s
Wall time: 23min 24s


In [21]:
# submission = np.random.randn(2, 5)
submission[1]

tensor([ 1.0000, -1.0484])

In [22]:
np.savez("2021-10-05-14-31-19_epoch_8_.npz", challenge_ids=submission[:, 0].int(), challenge_energy=submission[:, 1])

In [ ]:
sub = np.load("2021-10-03-20-12-13_SpinConv_SolidAngles_submission.npz")
sub['challenge_ids'][1]

In [ ]:
sub['challenge_energy'][1]